In [1]:
from infinipy.stateblock import StateBlock
import uuid

from infinipy.statement import Statement,CompositeStatement
from infinipy.transformer import Transformer,CompositeTransformer
from infinipy.affordance import Affordance
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any, Union, Tuple
from infinipy.options import Option
from infinipy.worldstatement import WorldStatement

In [2]:
#we define multiple entities we avoid the spatial component for now
# character 
# door
# key

@dataclass
class Door(StateBlock):
    is_open: bool = False  # Attribute to indicate if the door is open
    is_locked: bool = False  # Attribute to indicate if the door is locked
    def __init__(self, key_id: str,is_open:bool,is_locked:bool, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.key_id = key_id  # Identifier of the key that can open the door
        self.is_open = is_open
        self.is_locked = is_locked

    def __post_init__(self):
        super().__post_init__()  # Call the post-init of StateBlock
        # Additional initialization can be added here if needed
doorkey = StateBlock(
    id=str(uuid.uuid4()),
    owner_id="key_owner",  # Identifier of the key's owner or location
    name="Key",
    blocks_move=False,  # A key doesn't block movement
    blocks_los=False,  # A key doesn't block line of sight
    can_store=True,
    can_be_stored=False,
    can_act=True,
    can_move=False,
    can_be_moved=True,
    position=(0, 0, 0),  # Position of the key in the environment
)

door = Door(
    id=str(uuid.uuid4()),
    owner_id="door_owner",  # Identifier of the door's owner or location
    name="Door",
    blocks_move=True,  # A closed door blocks movement
    blocks_los=False,  # A door doesn't generally block line of sight
    can_store=False,
    can_be_stored=False,
    can_act=False,
    can_move=False,
    can_be_moved=False,
    position=(0, 0, 0),  # Position of the door in the environment
    is_open=False,  # Custom attribute indicating if the door is open
    is_locked=True,  # Custom attribute indicating if the door is locked
    key_id=doorkey.id,  # Custom attribute indicating the key that can open the door
)

character = StateBlock(
    id=str(uuid.uuid4()),
    owner_id="character_owner",  # Identifier of the character's owner or location
    name="Character",
    blocks_move=True,  # A character blocks movement
    blocks_los=True,  # A character blocks line of sight
    can_store=True,
    can_be_stored=False,
    can_act=True,
    can_move=True,
    can_be_moved=False,
    position=(0, 0, 0),  # Position of the character in the environment
)



In [3]:
#boolean conditions of the entities 

def is_open_condition(door: StateBlock):
    return door.is_open
def is_locked_condition(door: StateBlock):
    return door.is_locked

def has_key_condition(character: StateBlock):
    key_id = doorkey.id
    for item in character.inventory:
        if item.id == key_id:
            return True
    return False
    

def is_pickable(key: StateBlock):
    return key.can_be_moved

def can_pick(chararacter:StateBlock):
    return chararacter.can_store

def has_inventory_space(character: StateBlock):
    return len(character.inventory) < character.inventory_size


In [4]:
#statements corresponding

statements = {
    "IsOpen": Statement(
    name="IsOpen",
    description="is open",
    callable=is_open_condition,
    usage="target"
    ),
    "IsLocked": Statement(
    name="IsLocked",
    description="is locked",
    callable=is_locked_condition,
    usage="target"),
    "HasKey": Statement(
    name="HasKey",
    description="has key",
    callable=has_key_condition,
    usage="source"),
    "IsPickable": Statement(
    name="IsPickable",
    description="is pickable",
    callable=is_pickable,
    usage="target"),
    "CanPick": Statement(
    name="CanPick",
    description="can pick",
    callable=can_pick,
    usage="source"),
    "HasInventorySpace": Statement(
    name="HasInventorySpace",
    description="has inventory space",
    callable=has_inventory_space,
    usage="source"),
}

In [5]:
from copy import deepcopy

In [6]:
statements["IsOpen"]==deepcopy(statements["IsOpen"])

True

In [7]:
from infinipy.actions import Action

In [8]:
open_door_action = Action(name= "character_open_door",
                          prerequisites= [CompositeStatement([(statements["IsOpen"],False), 
                                                              (statements["IsLocked"],False)])],
                          consequences= [CompositeStatement([(statements["IsOpen"],True)])],
                          source_block= character,
                          target_block= door)

close_door_action = Action(name= "character_close_door",
                            prerequisites= [CompositeStatement([(statements["IsOpen"],True),
                                                                 (statements["IsLocked"],False)])],
                            consequences= [CompositeStatement([(statements["IsOpen"],False)])],
                            source_block= character,
                            target_block= door)

lock_door_action = Action(name= "character_lock_door",
                            prerequisites= [CompositeStatement([(statements["IsLocked"],False)]),
                                            CompositeStatement([(statements["HasKey"],True)])],
                            consequences= [CompositeStatement([(statements["IsLocked"],True)]),],
                            source_block= character,
                            target_block= door)

unlock_door_action = Action(name= "character_unlock_door",
                            prerequisites= [CompositeStatement([(statements["IsLocked"],True)]),
                                            CompositeStatement([(statements["HasKey"],True)])],
                            consequences= [CompositeStatement([(statements["IsLocked"],False)])],
                            source_block= character,
                            target_block= door)
pick_key_action = Action(name= "character_pick_key",
                            prerequisites= [CompositeStatement([(statements["IsPickable"],True)]),
                                            CompositeStatement([(statements["HasInventorySpace"],True)]),],
                            consequences= [CompositeStatement([(statements["IsPickable"],False)]),
                                           CompositeStatement([(statements["HasKey"],True)])],
                            source_block= character,
                            target_block= doorkey)
drop_key_action = Action(name= "character_drop_key",
                            prerequisites= [CompositeStatement([(statements["HasKey"],True)]),],
                            consequences= [CompositeStatement([(statements["IsPickable"],True)]),
                                           CompositeStatement([(statements["HasKey"],False)])],
                            source_block= character,
                            target_block= doorkey)
actions = [open_door_action,close_door_action,lock_door_action,unlock_door_action,pick_key_action]#,drop_key_action]


In [9]:
def get_forward_solution():
    option = Option()
    option.append(pick_key_action)
    option.append(unlock_door_action)
    option.append(open_door_action)
    return option
def get_backward_solution():
    backward_option = Option()
    backward_option.prepend(open_door_action)
    backward_option.prepend(unlock_door_action)
    backward_option.prepend(pick_key_action)
    return backward_option
def print_conditions(cond_dict : dict, with_key = False):
    for key,value in cond_dict.items():
        if with_key:
            print(key)
        print(value.name)

In [10]:
#starting statements
#the door is closed and locked
#the key is pickable
#the character has inventory space

statement_tuples=[(CompositeStatement([(statements["IsOpen"],False), (statements["IsLocked"],True)]),None,door.id),
                  (CompositeStatement([(statements["IsPickable"],True)]),None,doorkey.id),
                  (CompositeStatement([(statements["HasInventorySpace"],True)]),character.id,None),
                  (CompositeStatement([(statements["HasKey"],False)]),character.id,None)]
starting_state = WorldStatement(statement_tuples)
goal = CompositeStatement([(statements["IsOpen"],True)])
goal_state = WorldStatement([(goal,None,door.id)])

In [11]:
goal_state.available_actions(actions,reverse=True)

[Action(name=character_open_door, prerequisites=<infinipy.statement.CompositeStatement object at 0x000001FFDA3678B0>, consequences=<infinipy.statement.CompositeStatement object at 0x000001FFDA3677C0>, source_block=StateBlock(id='6c854750-4150-4897-85fc-32596c68ed3f', owner_id='character_owner', name='Character', blocks_move=True, blocks_los=True, can_store=True, can_be_stored=False, can_act=True, can_move=True, can_be_moved=False, position=(0, 0, 0), inventory=[], inventory_size=10, stored_in=None), target_block=Door(id='88122742-9003-4232-8cba-79f7c34a0203', owner_id='door_owner', name='Door', blocks_move=True, blocks_los=False, can_store=False, can_be_stored=False, can_act=False, can_move=False, can_be_moved=False, position=(0, 0, 0), inventory=[], inventory_size=10, stored_in=None, is_open=False, is_locked=True)),
 Action(name=character_lock_door, prerequisites=<infinipy.statement.CompositeStatement object at 0x000001FFDA366590>, consequences=<infinipy.statement.CompositeStatement o

In [12]:
from infinipy.goap import GOAP
# actions_out = goal_state.available_actions(actions, reverse=True)
# for action in actions_out:
#     print(action.name)
goap = GOAP (actions)
goap.backward_solve(goal_state,starting_state)

Trying action: character_open_door
State falsifies global consequences
CompositeStatement(IsOpen_target False, IsLocked_target False)
Trying action: character_lock_door
New available actions: ['character_open_door', 'character_close_door', 'character_unlock_door', 'character_pick_key']
Current path: character_lock_door
Trying action: character_open_door
State falsifies global consequences
CompositeStatement(HasKey_source True)
CompositeStatement(IsOpen_target False, IsLocked_target False)
Trying action: character_close_door
State falsifies global consequences
CompositeStatement(HasKey_source True)
CompositeStatement(IsOpen_target True, IsLocked_target False)
Trying action: character_unlock_door
New available actions: ['character_lock_door', 'character_pick_key']
Current path: character_lock_door <- character_unlock_door
Trying action: character_lock_door
State falsifies global consequences
CompositeStatement(HasKey_source True)
CompositeStatement(IsLocked_target False)
Trying action: c

In [13]:
goap.solutions

[[Action(name=character_pick_key, prerequisites=<infinipy.statement.CompositeStatement object at 0x000001FFDA364790>, consequences=<infinipy.statement.CompositeStatement object at 0x000001FFDA3648B0>, source_block=StateBlock(id='6c854750-4150-4897-85fc-32596c68ed3f', owner_id='character_owner', name='Character', blocks_move=True, blocks_los=True, can_store=True, can_be_stored=False, can_act=True, can_move=True, can_be_moved=False, position=(0, 0, 0), inventory=[], inventory_size=10, stored_in=None), target_block=StateBlock(id='91982587-4f7e-4b86-98a1-ffb70889dc95', owner_id='key_owner', name='Key', blocks_move=False, blocks_los=False, can_store=True, can_be_stored=False, can_act=True, can_move=False, can_be_moved=True, position=(0, 0, 0), inventory=[], inventory_size=10, stored_in=None)),
  Action(name=character_unlock_door, prerequisites=<infinipy.statement.CompositeStatement object at 0x000001FFDA366B30>, consequences=<infinipy.statement.CompositeStatement object at 0x000001FFDA366C8

In [14]:
goap.forward_solve(starting_state,goal_state)

Trying action: character_pick_key
Trying action: character_unlock_door


In [15]:
end_world = goap.terminal_world
goap.forward_solve(end_world,starting_state)

AttributeError: 'NoneType' object has no attribute 'available_actions'

In [ ]:
goap.backward_solve(goal_state,starting_state)

Trying action: character_open_door
State falsifies start state
CompositeStatement(IsOpen_target False, IsLocked_target False)
Trying action: character_lock_door
New available actions: ['character_open_door', 'character_close_door', 'character_unlock_door', 'character_pick_key']
Current path: character_lock_door
Trying action: character_open_door
State falsifies start state
CompositeStatement(HasKey_source True)
CompositeStatement(IsOpen_target False, IsLocked_target False)
Trying action: character_close_door
New available actions: ['character_open_door', 'character_unlock_door', 'character_pick_key']
Current path: character_lock_door <- character_close_door
Trying action: character_open_door
State falsifies start state
CompositeStatement(HasKey_source True)
CompositeStatement(IsOpen_target False, IsLocked_target False)
Trying action: character_unlock_door
New available actions: ['character_lock_door', 'character_pick_key']
Current path: character_lock_door <- character_close_door <- ch